In [8]:
import pandas as pd
import os
import json
import xmltodict
import json
import xml.etree.ElementTree as ET



In [9]:
# save file as json to read it manually

def convert_xml_to_json(xml_file, json_file):
    with open(xml_file, 'r') as file:
        xml_content = file.read()
    
    # Convert XML to a dictionary
    data_dict = xmltodict.parse(xml_content)
    
    # Convert dictionary to JSON
    json_data = json.dumps(data_dict, indent=4)
    
    # Write JSON data to a file
    with open(json_file, 'w') as file:
        file.write(json_data)
    
    print(f"Converted {xml_file} to {json_file}")

# Specify the input XML file and output JSON file
xml_file = 'data/Lesson 6.dtsx'
json_file = 'data/output.json'

convert_xml_to_json(xml_file, json_file)


Converted data/Lesson 6.dtsx to data/output.json


In [10]:
class Load():
    def __init__(self, path):
        self.path = path


    def remove_at_signs(self, obj):
        if isinstance(obj, dict):
            return {key.replace('@', ''): self.remove_at_signs(value) for key, value in obj.items()}
        elif isinstance(obj, list):
            return [self.remove_at_signs(item) for item in obj]
        else:
            return obj
        

    def remove_first_layer(self, json_dict):
        # Extract values from the first layer
        values = list(json_dict.values())
        return values

    def run(self):

        # Path to the XML file
        with open(self.path, 'rb') as f:
            self.xml = f.read()

   
        # open the xml file
        o = xmltodict.parse(self.xml)  # every time you reload the file in colab the key changes (file (1).xml becomes file (2).xml ...)

        json = self.remove_first_layer(self.remove_at_signs(o))[0]

        return json
    


files = [file for file in os.listdir('data/')]
#files = [file for file in os.listdir('data/Creating a Simple ETL Package/Completed Packages/')]

nodes_list = []
filter_list = []

for file in files[:3]:
    

    if file.endswith('.dtsx'):
        print(file)
        json = Load(f"data/{file}").run()
        #json = Load(f'data/Creating a Simple ETL Package/Completed Packages/{file}').run()
    #print(json)


DailyETLMain.dtsx
import_sql_flowA.dtsx


In [11]:
# parse sequence nodes
for sequence in json['DTS:Executable']:
    print(sequence)
    for node in sequence['DTS:Executable']:
        print("    ", node)
    print()



{'DTS:ExecutableType': 'STOCK:SEQUENCE', 'DTS:Property': [{'DTS:Name': 'LockName'}, {'DTS:Name': 'ForceExecValue', '#text': '0'}, {'DTS:Name': 'ExecValue', 'DTS:DataType': '3', '#text': '0'}, {'DTS:Name': 'ForceExecutionResult', '#text': '-1'}, {'DTS:Name': 'Disabled', '#text': '0'}, {'DTS:Name': 'FailPackageOnFailure', '#text': '0'}, {'DTS:Name': 'FailParentOnFailure', '#text': '0'}, {'DTS:Name': 'MaxErrorCount', '#text': '1'}, {'DTS:Name': 'ISOLevel', '#text': '1048576'}, {'DTS:Name': 'LocaleID', '#text': '-1'}, {'DTS:Name': 'TransactionOption', '#text': '1'}, {'DTS:Name': 'DelayValidation', '#text': '0'}, {'DTS:Name': 'ObjectName', '#text': 'Incassi - Import from Oracle (SUN)'}, {'DTS:Name': 'DTSID', '#text': '{d25b4111-27b6-4bab-a712-df49b7b44de3}'}, {'DTS:Name': 'Description', '#text': 'Sequence Container'}, {'DTS:Name': 'CreationName', '#text': 'STOCK:SEQUENCE'}, {'DTS:Name': 'DisableEventHandlers', '#text': '0'}], 'DTS:LoggingOptions': {'DTS:Property': [{'DTS:Name': 'LoggingMode

In [12]:
# parse names nodes and type

for sequence in json['DTS:Executable']:

    for i in range(sequence['DTS:Executable'].__len__()):
        print(sequence['DTS:Executable'][i]['DTS:ExecutableType'])

        for property in sequence['DTS:Executable'][i]['DTS:Property']:
            if "ObjectName" in str(property):
                print('name: ', property['#text'])
            if "Description" in str(property):
                print('type: ', property['#text'])
            
          

        
        # sql script (executesqltask)
        try:
        
            print("execute sql command: ", sequence['DTS:Executable'][i]['DTS:ObjectData']['SQLTask:SqlTaskData']['SQLTask:SqlStatementSource'])
        except:
            pass

        # sql script (scripttask)
        try:
            print(sequence['DTS:Executable'][i]['DTS:ObjectData']['ScriptProject']['ProjectItem'][0]['#text'])
        except:
            pass

        # sql script (data flow task)
        try:
            print(sequence['DTS:Executable'][i]['DTS:ObjectData']['pipeline']['components']['component'][0]['properties']['property'][3]['#text'])
        except:
            pass

        print()
        print('################################')
        print()
    print()

Microsoft.SqlServer.Dts.Tasks.ScriptTask.ScriptTask, Microsoft.SqlServer.ScriptTask, Version=10.0.0.0, Culture=neutral, PublicKeyToken=89845dcd8080cc91
name:  SCT-1 Preapare Oracle SQL Statement
type:  Script Task
/*
   Microsoft SQL Server Integration Services Script Task
   Write scripts using Microsoft Visual C# 2008.
   The ScriptMain is the entry point class of the script.
*/

using System;
using System.Data;
using Microsoft.SqlServer.Dts.Runtime;
using System.Windows.Forms;

namespace ST_d59b322e25fb4079bd92bee984cda257.csproj
{
    [System.AddIn.AddIn("ScriptMain", Version = "1.0", Publisher = "", Description = "")]
    public partial class ScriptMain : Microsoft.SqlServer.Dts.Tasks.ScriptTask.VSTARTScriptObjectModelBase
    {

        #region VSTA generated code
        enum ScriptResults
        {
            Success = Microsoft.SqlServer.Dts.Runtime.DTSExecResult.Success,
            Failure = Microsoft.SqlServer.Dts.Runtime.DTSExecResult.Failure
        };
        #endregion

In [13]:
# parse c# code in nodes

import re


def extract_sql_query(script):
    pattern = re.compile(r"return\s+@\"(.*?)\";", re.DOTALL)
    match = pattern.search(script)
    
    if match:
        raw_query = match.group(1)
        return raw_query
    else:
        return None
    
queries = []

for sequence in json['DTS:Executable']:
    for node in sequence['DTS:Executable']:
        try:
            #print(node['DTS:ObjectData']['ScriptProject']['ProjectItem'][0]['#text']) # extract c# with sql inside
            queries.append(extract_sql_query(node['DTS:ObjectData']['ScriptProject']['ProjectItem'][0]['#text'])) # extract sql from c# code

        except:
            pass
    print()



In [14]:
queries

["\nSELECT  quietanza.id_quietanza as chiave,\n\tcase\n\twhen quietanza.id_tipo_modello = 1 then 1\n\telse 2\n\tend as numtipomodello,\n\ttrim( cast( generelocale.indicatore_se as varchar(1) ) ) as indicatorese,\n\tquietanza.NUM_FATTURA as numeroFattura,\n\tquietanza.DATA as dataQuietanza,\n\t\n\tspazio.DENOMINAZIONE as denominazioneSpazio,\n\t\n\ttoponimoLocale.DENOMINAZIONE as TOPONIMOLOCALE,\n\tstradaLocale.DENOMINAZIONE as STRADALOCALE,\n\tlocale.NUMERO_CIVICO as NUMEROCIVICO,\n\ttrim( cast( comuneLocale.COD_COMUNE_SIAE as varchar(3) ) ) as codiceComuneSiae,\n\ttrim( cast( locale.COD_CATEG_LOCALE as varchar(1) ) ) as CODCATEGLOCALE,\n\ttrim( cast( locale.COD_GENERE_LOCALE as varchar(3) ) ) as genereLocale,\n\torganizzatore.COGNOME as cognomeOrganizzatore,\n\torganizzatore.NOME as nomeOrganizzatore,\n\torganizzatore.CODICE_FISCALE as CODICEFISCALE,\n\torganizzatore.PARTITA_IVA as PARTITAIVA,\n\t\n\tstradaOrganizzatore.ID_STRADA as IDSTRADAORGANIZZATORE,\n\ttoponimoOrganizzatore.DENO

In [15]:
# local imports
import sys
sys.path.append(r"C:\Users\PietroGarroni\projects\RABO")
from modules.sql_parser.parse_nodes import *


In [16]:
from sqlglot import parse_one, exp
from sqlglot.dialects.tsql import TSQL
import pypyodbc as odbc
import configparser
import copy
from collections import defaultdict, OrderedDict
import pandas as pd
import configparser
import os
import json
import re

In [17]:
query = queries[0]

print(query)


SELECT  quietanza.id_quietanza as chiave,
	case
	when quietanza.id_tipo_modello = 1 then 1
	else 2
	end as numtipomodello,
	trim( cast( generelocale.indicatore_se as varchar(1) ) ) as indicatorese,
	quietanza.NUM_FATTURA as numeroFattura,
	quietanza.DATA as dataQuietanza,
	
	spazio.DENOMINAZIONE as denominazioneSpazio,
	
	toponimoLocale.DENOMINAZIONE as TOPONIMOLOCALE,
	stradaLocale.DENOMINAZIONE as STRADALOCALE,
	locale.NUMERO_CIVICO as NUMEROCIVICO,
	trim( cast( comuneLocale.COD_COMUNE_SIAE as varchar(3) ) ) as codiceComuneSiae,
	trim( cast( locale.COD_CATEG_LOCALE as varchar(1) ) ) as CODCATEGLOCALE,
	trim( cast( locale.COD_GENERE_LOCALE as varchar(3) ) ) as genereLocale,
	organizzatore.COGNOME as cognomeOrganizzatore,
	organizzatore.NOME as nomeOrganizzatore,
	organizzatore.CODICE_FISCALE as CODICEFISCALE,
	organizzatore.PARTITA_IVA as PARTITAIVA,
	
	stradaOrganizzatore.ID_STRADA as IDSTRADAORGANIZZATORE,
	toponimoOrganizzatore.DENOMINAZIONE as TOPONIMOORGANIZZATORE,
	stradaOrgani

In [29]:
query = queries[0]

ast = parse_query(query) # get parsed tree

tree = replace_aliases(query) # get transformed tree without table aliases


# parse selects before getting statements
select_statement = tree.find(exp.Select)

tree


Select(
  expressions=[
    Alias(
      this=Column(
        this=Identifier(this=id_quietanza, quoted=False),
        table=Identifier(this=QUIETANZA, quoted=False)),
      alias=Identifier(this=chiave, quoted=False)),
    Alias(
      this=Case(
        ifs=[
          If(
            this=EQ(
              this=Column(
                this=Identifier(this=id_tipo_modello, quoted=False),
                table=Identifier(this=QUIETANZA, quoted=False)),
              expression=Literal(this=1, is_string=False)),
            true=Literal(this=1, is_string=False))],
        default=Literal(this=2, is_string=False)),
      alias=Identifier(this=numtipomodello, quoted=False)),
    Alias(
      this=Trim(
        this=Cast(
          this=Column(
            this=Identifier(this=indicatore_se, quoted=False),
            table=Identifier(this=generelocale, quoted=False)),
          to=DataType(
            this=Type.VARCHAR,
            expressions=[
              DataTypeParam(
           

In [47]:
print(ast)

SELECT quietanza.id_quietanza AS chiave, CASE WHEN quietanza.id_tipo_modello = 1 THEN 1 ELSE 2 END AS numtipomodello, TRIM(CAST(generelocale.indicatore_se AS VARCHAR(1))) AS indicatorese, quietanza.NUM_FATTURA AS numeroFattura, quietanza.DATA AS dataQuietanza, spazio.DENOMINAZIONE AS denominazioneSpazio, toponimoLocale.DENOMINAZIONE AS TOPONIMOLOCALE, stradaLocale.DENOMINAZIONE AS STRADALOCALE, locale.NUMERO_CIVICO AS NUMEROCIVICO, TRIM(CAST(comuneLocale.COD_COMUNE_SIAE AS VARCHAR(3))) AS codiceComuneSiae, TRIM(CAST(locale.COD_CATEG_LOCALE AS VARCHAR(1))) AS CODCATEGLOCALE, TRIM(CAST(locale.COD_GENERE_LOCALE AS VARCHAR(3))) AS genereLocale, organizzatore.COGNOME AS cognomeOrganizzatore, organizzatore.NOME AS nomeOrganizzatore, organizzatore.CODICE_FISCALE AS CODICEFISCALE, organizzatore.PARTITA_IVA AS PARTITAIVA, stradaOrganizzatore.ID_STRADA AS IDSTRADAORGANIZZATORE, toponimoOrganizzatore.DENOMINAZIONE AS TOPONIMOORGANIZZATORE, stradaOrganizzatore.DENOMINAZIONE AS STRADAORGANIZZATORE,

In [33]:
on_condition = tree.find_all(exp.EQ)
for i in on_condition:
    print(i)

QUIETANZA.ID_INFO_STAMPA_QUIETANZA = INFO_STAMPA_QUIETANZA.ID_INFO_STAMPA_QUIETANZA
SPAZIO.id_spazio = QUIETANZA.id_spazio
SPAZIO.ID_SPAZIO = RAGGRUPPAMENTO_SPAZIO.ID_SPAZIO
RAGGRUPPAMENTO_SPAZIO.ID_RAGGRUPPAMENTO = RAGGRUPPAMENTO.ID_RAGGRUPPAMENTO
SPAZIO.ID_LOCALE = LOCALE.ID_LOCALE
ORGANIZZATORE.ID_STRADA = STRADA.ID_STRADA
STRADA.COD_TOPONIMO = TOPONIMO.COD_TOPONIMO
STRADA.COD_COMUNE = COMUNE.COD_COMUNE
COMUNE.COD_PROVINCIA = PROVINCIA.COD_PROVINCIA
SPAZIO.ID_SPAZIO = CODICE_SPEI.ID_SPAZIO
QUIETANZA.COD_SEPRAG = CODICE_SPEI.COD_SEPRAG
QUIETANZA.id_tipo_modello = 1
LOCALE.cod_genere_locale = generelocale.cod_genere_locale
TIPO_DOCUMENTO.ID_TIPO_DOCUMENTO = QUIETANZA.ID_TIPO_DOCUMENTO
STRADA.COD_COMUNE = COMUNE.COD_COMUNE
STRADA.COD_TOPONIMO = TOPONIMO.COD_TOPONIMO
LOCALE.ID_STRADA = STRADA.ID_STRADA
QUIETANZA.ID_ORGANIZZATORE = ORGANIZZATORE.ID_ORGANIZZATORE
QUIETANZA.origine = 'SUN'


In [23]:



source_table, where_exp = get_statements(select_statement) 

print(source_table)

source_table = get_tables(select_statement) 

print(source_table)

join_statements = tree.find_all(exp.Join)



from_statements = tree.find_all(exp.From)
for i in from_statements:
    print(i)

#on_condition = on_statement(select_statement)

['QUIETANZA', 'INFO_STAMPA_QUIETANZA']
[('QUIETANZA', 'quietanza'), ('INFO_STAMPA_QUIETANZA', 'infoStampaQuietanza'), ('SPAZIO', 'spazio'), ('RAGGRUPPAMENTO_SPAZIO', 'raggruppamentoSpazio'), ('RAGGRUPPAMENTO', 'raggruppamento'), ('LOCALE', 'locale'), ('CODICE_SPEI', 'codiceSpei'), ('ORGANIZZATORE', 'organizzatore'), ('STRADA', 'stradaOrganizzatore'), ('TOPONIMO', 'toponimoOrganizzatore'), ('COMUNE', 'comuneOrganizzatore'), ('PROVINCIA', 'provincia'), ('STRADA', 'stradaLocale'), ('TOPONIMO', 'toponimoLocale'), ('COMUNE', 'comuneLocale'), ('TIPO_DOCUMENTO', 'tipoDocumento'), ('GENERE_LOCALE', 'genereLocale')]
FROM QUIETANZA AS quietanza


In [21]:
for i in tree.find_all(exp.Table):
    print(i)

QUIETANZA AS quietanza
INFO_STAMPA_QUIETANZA AS infoStampaQuietanza
SPAZIO AS spazio
RAGGRUPPAMENTO_SPAZIO AS raggruppamentoSpazio
RAGGRUPPAMENTO AS raggruppamento
LOCALE AS locale
CODICE_SPEI AS codiceSpei
ORGANIZZATORE AS organizzatore
STRADA AS stradaOrganizzatore
TOPONIMO AS toponimoOrganizzatore
COMUNE AS comuneOrganizzatore
PROVINCIA AS provincia
STRADA AS stradaLocale
TOPONIMO AS toponimoLocale
COMUNE AS comuneLocale
TIPO_DOCUMENTO AS tipoDocumento
GENERE_LOCALE AS genereLocale


In [24]:
for i in tree.find_all(exp.Where):
    print(i)

WHERE QUIETANZA.STATO IN ('S', 'A') AND QUIETANZA.origine = 'SUN' AND QUIETANZA.ID_ORGANIZZATORE = ORGANIZZATORE.ID_ORGANIZZATORE AND LOCALE.ID_STRADA = STRADA.ID_STRADA AND STRADA.COD_TOPONIMO = TOPONIMO.COD_TOPONIMO AND STRADA.COD_COMUNE = COMUNE.COD_COMUNE AND TIPO_DOCUMENTO.ID_TIPO_DOCUMENTO = QUIETANZA.ID_TIPO_DOCUMENTO AND LOCALE.cod_genere_locale = generelocale.cod_genere_locale AND QUIETANZA.data >= TO_DATE(STRUCT(0), STRUCT(2)) AND QUIETANZA.data < TO_DATE(STRUCT(1), STRUCT(2))


In [45]:
# join arguments

for i in tree.find_all(exp.Join):
    print(i)
    print(i.side)
    print(i.kind)
    print(i.find(exp.EQ))
    print('##################')

LEFT JOIN INFO_STAMPA_QUIETANZA AS infoStampaQuietanza ON QUIETANZA.ID_INFO_STAMPA_QUIETANZA = INFO_STAMPA_QUIETANZA.ID_INFO_STAMPA_QUIETANZA
LEFT

QUIETANZA.ID_INFO_STAMPA_QUIETANZA = INFO_STAMPA_QUIETANZA.ID_INFO_STAMPA_QUIETANZA
##################
INNER JOIN SPAZIO AS spazio ON SPAZIO.id_spazio = QUIETANZA.id_spazio

INNER
SPAZIO.id_spazio = QUIETANZA.id_spazio
##################
LEFT JOIN RAGGRUPPAMENTO_SPAZIO AS raggruppamentoSpazio ON SPAZIO.ID_SPAZIO = RAGGRUPPAMENTO_SPAZIO.ID_SPAZIO
LEFT

SPAZIO.ID_SPAZIO = RAGGRUPPAMENTO_SPAZIO.ID_SPAZIO
##################
LEFT JOIN RAGGRUPPAMENTO AS raggruppamento ON RAGGRUPPAMENTO_SPAZIO.ID_RAGGRUPPAMENTO = RAGGRUPPAMENTO.ID_RAGGRUPPAMENTO
LEFT

RAGGRUPPAMENTO_SPAZIO.ID_RAGGRUPPAMENTO = RAGGRUPPAMENTO.ID_RAGGRUPPAMENTO
##################
INNER JOIN LOCALE AS locale ON SPAZIO.ID_LOCALE = LOCALE.ID_LOCALE

INNER
SPAZIO.ID_LOCALE = LOCALE.ID_LOCALE
##################
LEFT JOIN CODICE_SPEI AS codiceSpei ON SPAZIO.ID_SPAZIO = CODICE_SPEI.ID_SPAZI

In [26]:
join_statements = tree.find_all(exp.Join)
for i in tree.find_all(exp.Join):
    print(i)
    print()

LEFT JOIN INFO_STAMPA_QUIETANZA AS infoStampaQuietanza ON QUIETANZA.ID_INFO_STAMPA_QUIETANZA = INFO_STAMPA_QUIETANZA.ID_INFO_STAMPA_QUIETANZA

INNER JOIN SPAZIO AS spazio ON SPAZIO.id_spazio = QUIETANZA.id_spazio

LEFT JOIN RAGGRUPPAMENTO_SPAZIO AS raggruppamentoSpazio ON SPAZIO.ID_SPAZIO = RAGGRUPPAMENTO_SPAZIO.ID_SPAZIO

LEFT JOIN RAGGRUPPAMENTO AS raggruppamento ON RAGGRUPPAMENTO_SPAZIO.ID_RAGGRUPPAMENTO = RAGGRUPPAMENTO.ID_RAGGRUPPAMENTO

INNER JOIN LOCALE AS locale ON SPAZIO.ID_LOCALE = LOCALE.ID_LOCALE

LEFT JOIN CODICE_SPEI AS codiceSpei ON SPAZIO.ID_SPAZIO = CODICE_SPEI.ID_SPAZIO AND QUIETANZA.COD_SEPRAG = CODICE_SPEI.COD_SEPRAG

, ORGANIZZATORE AS organizzatore

LEFT JOIN STRADA AS stradaOrganizzatore ON ORGANIZZATORE.ID_STRADA = STRADA.ID_STRADA

LEFT JOIN TOPONIMO AS toponimoOrganizzatore ON STRADA.COD_TOPONIMO = TOPONIMO.COD_TOPONIMO

LEFT JOIN COMUNE AS comuneOrganizzatore ON STRADA.COD_COMUNE = COMUNE.COD_COMUNE

LEFT JOIN PROVINCIA AS provincia ON COMUNE.COD_PROVINCIA = 

In [16]:
query = queries[0]

nodes = []


if query.startswith("("): # remove open and closing paranthesis from subqueries
    query = query.strip("()")
else:
    pass

ast = parse_query(query) # get parsed tree

tree = replace_aliases(query) # get transformed tree without table aliases

# parse selects before getting statements
select_statement_big = tree.find_all(exp.Select)

source_tables = []
# for every select statement in query, extract the source tables, where expressions and on conditions
for select in list(select_statement_big):
    
    source_table, where_exp = get_statements(select) 

    on_condition = on_statement(select)

    source_tables += source_table


    nodes.append({'NAME_NODE': "target_node", 'LABEL_NODE': f'{"filename"}@{"target_node"}', 'WHERE': where_exp, 'FUNCTION': 'subquery', 'ON': on_condition})
else:
    try: # try to find the create or insert into statements
        #target_node = list(ast.find_all(exp.Create))[0].this.this.this
        nodes.append({'NAME_NODE': 'target_node','LABEL_NODE': 'target_node', 'WHERE': where_exp, 'FUNCTION': 'target', 'ON': on_condition})
    except IndexError:
        #target_node = list(ast.find_all(exp.Insert))[0].this.this
        nodes.append({'NAME_NODE': 'target_node','LABEL_NODE': 'target_node', 'WHERE': where_exp, 'FUNCTION': 'target', 'ON': on_condition})


for table in source_tables:
    if table not in [node['NAME_NODE'] for node in nodes]:
        if 'subquery' in table: 
            nodes.append({'NAME_NODE': table,'LABEL_NODE': f'{"filename"}@{table}', 'WHERE': None, 'FUNCTION': 'subquery', 'ON': on_condition})
        else:
            nodes.append({'NAME_NODE': table,'LABEL_NODE': table, 'WHERE': None, 'FUNCTION': 'table', 'ON': None})

In [19]:
print(query)


SELECT  quietanza.id_quietanza as chiave,
	case
	when quietanza.id_tipo_modello = 1 then 1
	else 2
	end as numtipomodello,
	trim( cast( generelocale.indicatore_se as varchar(1) ) ) as indicatorese,
	quietanza.NUM_FATTURA as numeroFattura,
	quietanza.DATA as dataQuietanza,
	
	spazio.DENOMINAZIONE as denominazioneSpazio,
	
	toponimoLocale.DENOMINAZIONE as TOPONIMOLOCALE,
	stradaLocale.DENOMINAZIONE as STRADALOCALE,
	locale.NUMERO_CIVICO as NUMEROCIVICO,
	trim( cast( comuneLocale.COD_COMUNE_SIAE as varchar(3) ) ) as codiceComuneSiae,
	trim( cast( locale.COD_CATEG_LOCALE as varchar(1) ) ) as CODCATEGLOCALE,
	trim( cast( locale.COD_GENERE_LOCALE as varchar(3) ) ) as genereLocale,
	organizzatore.COGNOME as cognomeOrganizzatore,
	organizzatore.NOME as nomeOrganizzatore,
	organizzatore.CODICE_FISCALE as CODICEFISCALE,
	organizzatore.PARTITA_IVA as PARTITAIVA,
	
	stradaOrganizzatore.ID_STRADA as IDSTRADAORGANIZZATORE,
	toponimoOrganizzatore.DENOMINAZIONE as TOPONIMOORGANIZZATORE,
	stradaOrgani

In [17]:
nodes

[{'NAME_NODE': 'target_node',
  'LABEL_NODE': 'filename@target_node',
  'WHERE': "QUIETANZA.STATO IN ('S', 'A') AND QUIETANZA.origine = 'SUN' AND QUIETANZA.ID_ORGANIZZATORE = ORGANIZZATORE.ID_ORGANIZZATORE AND LOCALE.ID_STRADA = STRADA.ID_STRADA AND STRADA.COD_TOPONIMO = TOPONIMO.COD_TOPONIMO AND STRADA.COD_COMUNE = COMUNE.COD_COMUNE AND TIPO_DOCUMENTO.ID_TIPO_DOCUMENTO = QUIETANZA.ID_TIPO_DOCUMENTO AND LOCALE.cod_genere_locale = generelocale.cod_genere_locale AND QUIETANZA.data >= TO_DATE(STRUCT(0), STRUCT(2)) AND QUIETANZA.data < TO_DATE(STRUCT(1), STRUCT(2))",
  'FUNCTION': 'subquery',
  'ON': []},
 {'NAME_NODE': 'target_node',
  'LABEL_NODE': 'target_node',
  'WHERE': "QUIETANZA.STATO IN ('S', 'A') AND QUIETANZA.origine = 'SUN' AND QUIETANZA.ID_ORGANIZZATORE = ORGANIZZATORE.ID_ORGANIZZATORE AND LOCALE.ID_STRADA = STRADA.ID_STRADA AND STRADA.COD_TOPONIMO = TOPONIMO.COD_TOPONIMO AND STRADA.COD_COMUNE = COMUNE.COD_COMUNE AND TIPO_DOCUMENTO.ID_TIPO_DOCUMENTO = QUIETANZA.ID_TIPO_DOCUMEN

In [ ]:
# create nodes

nodes_dfs = []

queries= []
for i, file in enumerate(os.listdir("data/preprocessed-queries/sqlglot-sql-queries-extracted3")):
    filename = file.split('.')[0]

    files.append(file.split('.')[0])
    with open(f'data/preprocessed-queries/sqlglot-sql-queries-extracted3/{file}', 'r') as file:
        data = json.load(file)

    queries.append(data)

    query_subqueries = dict(reversed(list(queries[i]['subquery_dictionary'].items())))
    query_subqueries['main_query'] = queries[i]['modified_SQL_query']

    nodes = []
    for name_query in query_subqueries:

        query = query_subqueries[name_query]
        if query.startswith("("): # remove open and closing paranthesis from subqueries
            query = query.strip("()")
        else:
            pass
        
        ast = parse_query(query) # get parsed tree
        
        tree = replace_aliases(query) # get transformed tree without table aliases
        
        # parse selects before getting statements
        select_statement_big = tree.find_all(exp.Select)

        source_tables = []
        # for every select statement in query, extract the source tables, where expressions and on conditions
        for select in list(select_statement_big):
            
            source_table, where_exp = get_statements(select) 

            on_condition = on_statement(select)

            source_tables += source_table

        if 'subquery' in name_query: # if the query is a subquery then the name is the dict key, else the name is the target table

            target_node = name_query
            nodes.append({'NAME_NODE': target_node, 'LABEL_NODE': f'{filename}@{target_node}', 'WHERE': where_exp, 'FUNCTION': 'subquery', 'ON': on_condition})
        else:
            try: # try to find the create or insert into statements
                target_node = list(ast.find_all(exp.Create))[0].this.this.this
                nodes.append({'NAME_NODE': target_node,'LABEL_NODE': target_node, 'WHERE': where_exp, 'FUNCTION': 'target', 'ON': on_condition})
            except IndexError:
                target_node = list(ast.find_all(exp.Insert))[0].this.this
                print(target_node)
                nodes.append({'NAME_NODE': target_node,'LABEL_NODE': target_node, 'WHERE': where_exp, 'FUNCTION': 'target', 'ON': on_condition})


        for table in source_tables:
            if table not in [node['NAME_NODE'] for node in nodes]:
                if 'subquery' in table: 
                    nodes.append({'NAME_NODE': table,'LABEL_NODE': f'{filename}@{table}', 'WHERE': None, 'FUNCTION': 'subquery', 'ON': on_condition})
                else:
                    nodes.append({'NAME_NODE': table,'LABEL_NODE': table, 'WHERE': None, 'FUNCTION': 'table', 'ON': None})
                    
    nodes = pd.DataFrame(nodes)
    nodes['COLOR'] = 'grey'

    nodes_dfs.append(pd.DataFrame(nodes))


# create and save dataframe
nodes = pd.concat(nodes_dfs).reset_index(drop=True)
nodes['ON'] = nodes['ON'].apply(lambda x: None if x == [] else x) # remove empty lists
nodes = nodes.drop_duplicates(subset=['NAME_NODE', 'LABEL_NODE', 'WHERE', 'FUNCTION']).reset_index(drop=True)
nodes['ID'] = nodes.index
